# Document Classification Project


In [2]:
# set up libraries 
import pandas as pd
import glob
import os


In [29]:
# Get data
df = pd.DataFrame(columns=['Text', 'Label'])
main_path = r'C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/'
end_path = ['Analytical_Report', 'Incident_Report', 'Situation_Report']
for end in end_path:
    path = main_path + end
    all_files = glob.glob(os.path.join(path, "*.txt")) 
    # print(all_files[0])
    data = [[f, end] for f in all_files]
    # print(pd.DataFrame(data, columns=['Text', 'Label']))
    df = df.append(pd.DataFrame(data, columns=['Text', 'Label']))
print(df)


                                                 Text              Label
0   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
1   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
2   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
3   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
4   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
..                                                ...                ...
65  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
66  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
67  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
68  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
69  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report

[157 rows x 2 columns]
